# Identifying Phase Transitions

In this notebook, we will use numerous methods to characterize the phase behavior of systems of hard polygons.
At low pressures, systems will be in a fluid phase.
At high enough pressures, the system will form a solid.
There is also an intermediate hexatic phase, but that takes more care and much larger simulations to identify - see [ Shape and Symmetry Determine Two-Dimensional Melting Transitions of Hard Regular Polygons](http://dx.doi.org/10.1103/PhysRevX.7.021001).

We start by importing the signac project and setting a few global variables.

In [ ]:
from project import Project

project = Project()

# If you only want to read/store data, you can also just do this:
# import signac
# project = signac.get_project()


# Specify the resolution for all figures below:
DPI = 140

## Analysis of a single State Point

As a first step, we will identify the state (fluid or solid), for a particular combination of parameters after the system has equilibrated.
We will perform various analyses using [Freud](glotzerlab.engin.umich.edu/freud) to show how we can identify the phase.
Once we have identified that this occurs for a some pressure for a given shape, we will look at multiple data points in concert to identify the critical point.

Each simulation is executed for a particular pressure ($betaP$), number of vertices ($n$), and a unique random seed.
This parameter space is managed with [signac](http://www.signac.io) and the *implicit* schema contains exactly these three variables:

In [ ]:
print(project.detect_schema())

We carry out our simulations and analyses for each unique set of parameters, and all the resulting data is stored in an associated directory.
In the context of **signac**, each unique set of parameters is called a *state point*, and the association of state point metadata and directory is called a *job*.
Each job represents a unique point in our parameter space and contains all the data we have collected for it.

To interact with our data in **signac**, we typically want to look at sets of jobs.
The standard interface for accessing job data is through the `find_jobs()` function, which lets us filter our data space for jobs matching a specific set of state point keys.

In [ ]:
print("The total number of data points we have is {}.".format(len(project.find_jobs())))
print("Of these, {} have parameter betaP=10.".format(len(project.find_jobs(dict(n=5, betaP=10.0)))))

Since the Python job object is simply a Python wrapper around a file path, we always have direct access to that file path; accessing this path, and any files on that path, is the simplest usage of a **signac** job.

In [ ]:
job = list(project.find_jobs(dict(n=5, betaP=10.0)))[0]

print("The data for one of the jobs with n=5 and betaP=10 is stored at this path: {}.\n".format(job.workspace()))
print("The state point is always readily accessible: {}.\n".format(job.sp))
print("We can create or access file paths within this workspace: {}.\n".format(job.fn('data.txt')))

In this notebook we will always inspect **exactly one** of these state points and we can select it interactively using ipywidgets.

**You only need to execute the cell below once. Once you execute it, simply choosing the appropriate parameters in the dropdown will pick new jobs for you interactively without running the code manually again.**

In [ ]:
import sys
from ipywidgets import interact

JOB = None

schema = project.detect_schema()
ns = schema['n'][int]
betaPs = schema['betaP'][float]
seeds = schema['seed'][int]

@interact(n=ns, betaP=betaPs, seed=seeds)
def select(n=5, betaP=13.2, seed=0):
    global JOB
    jobs = project.find_jobs(dict(n=n, betaP=betaP, seed=seed))
    if len(jobs) == 0:
        print("No jobs found for this selection.", file=sys.stderr)
    elif len(jobs) > 1:
        print("Multiple jobs matched this selection.", file=sys.stderr)
    else:
        JOB = list(jobs)[0]
        print("Selected job with id:", JOB)

Use the widget above to select a specific state point to examine.
Then, run (or re-run) the cells below to examine the results from that particular simulation and analysis.

### Equilibration

As a first step, we plot the average hexatic order parameter and system density over time. 
Over the course of the simulation, these quantities will stabilize as the system equilibrates.

The [k-atic order parameter](https://freud.readthedocs.io/en/stable/order.html#freud.order.HexOrderParameter) measures *k-fold* order in a system of particles by calculating the angles between a particle and its neighbors.
Here, we calculate $\psi$, the magnitude of the average 6-atic, or **hexatic**, order parameter, which measures the 6-fold order.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import gsd.hoomd
%matplotlib inline

with gsd.hoomd.open(JOB.fn('trajectory.gsd')) as traj:
    N = traj[-1].particles.N

log = np.genfromtxt(fname=JOB.fn('log.dat'), names=True)
psi = np.load(JOB.fn('order.npz'))

fig, ax = plt.subplots(dpi=DPI)
ax2 = ax.twinx()

ax.plot(log['timestep'], N * JOB.doc.poly_area / log['volume'])
ax.set_xlabel('time step')
ax.set_ylabel('Packing Fraction')

ax2.plot(psi['steps'], np.absolute(psi['psi'].mean(axis=1)), color='red')
ax2.set_ylabel('Hexatic Order Parameter ($\psi$)', color='red')

plt.show()

Fluids and solids will show different behavior:
* **Fluid:** Low pressure fluids will exhibit very little hexatic order, the equilibrium $\psi$ will be very small.
* **Solid:** In ordered solids, $\psi$ will be large, approximately 0.5 and greater.

### Hexatic Order Parameter

We can also visualize the system directly, with each particle colored by the value of the local hexatic order parameter.
Note that instead of using the stored values, we are now using [`freud`](https://freud.readthedocs.io/en/stable/order.html#freud.order.HexOrderParameter) to calculate the order parameter on the fly each time we visualize a new frame.

In [ ]:
import freud
from draw_utils import quat2ang, draw_config, draw_pmft, draw_voronoi

with gsd.hoomd.open(JOB.fn('trajectory.gsd')) as traj:
    num_frames = len(traj)-1 

In [ ]:
@interact(frame=(1, num_frames))
def frame_demo(frame=num_frames):
    fix, ax = plt.subplots(figsize=(6, 6), dpi=DPI)
    with gsd.hoomd.open(JOB.fn('trajectory.gsd')) as traj:
        frame = traj[frame]
        
        box = freud.box.Box.from_box(frame.configuration.box[:2].tolist())
        hop = freud.order.HexOrderParameter(rmax=1.2, k=6)
        hop.compute(box, frame.particles.position)
        
        draw_config(fig, ax, box, frame.particles.position, quat2ang(frame.particles.orientation), hop.psi, JOB.sp.n)

Fluids and solids will show different behavior:
* **Fluid:** Fluids have very little correlation in hexatic order. Particles will show the full range of the color wheel:
* **Solid:** In ordered solids, all the local environments are ordered. Particles will most all be the same color (except in a few local regions near defects).

### Voronoi Diagram

Another way to count defects is to look at the [Voronoi diagram](https://en.wikipedia.org/wiki/Voronoi_diagram) of the system.
Using the particle positions as reference points, the Voronoi diagram partitions space and finds every point is associated with the closest reference point.
In a **perfectly ordered system**, we would expect the Voronoi diagram to divide space into perfect hexagons.
Anywhere there are defects in the system, however, we would expect the Voronoi diagram to show pentagons and heptagons (and occasionally larger polygons).
To demonstrate this, we plot the Voronoi diagrams below, with each Voronoi polygon colored by the number of sides.

In [ ]:
@interact(frame=(1, num_frames))
def voronoi_demo(frame=num_frames):
    with gsd.hoomd.open(JOB.fn('trajectory.gsd')) as traj:
        frame = traj[frame]
            
        box = freud.box.Box.from_box(frame.configuration.box[:2].tolist())
        voronoi = freud.voronoi.Voronoi(box, np.sqrt(box.Lx**2 + box.Ly**2)/5)
        voronoi.compute(frame.particles.position, box)

    fig, ax = plt.subplots(1, 1, figsize=(10, 8), dpi=DPI)
    draw_voronoi(fig, ax, box, voronoi.polytopes)

As we see, early on in the simulation that there are many 5- and 7-gons, as the system is still equilibrating.
When equilibrated, you should see:

* **Fluid**: Mostly hexagons, but a high concentration of 5- and 7-gons connected to each other.
* **Solid**: Mostly hexagons, with a few 5- and 7-gons connected to each other.

Solids in 2D only have "quasi" long range order due to an appreciable concentration of defects at equilibrium. Compare the Voronoi diagram to the same frame above with the hexatic order parameter coloring and the defects should be visible in the same locations.

### Radial Distribution Function (RDF)

Another way we can analyze our ordering is using the [radial distribution function (RDF)](https://en.wikipedia.org/wiki/Radial_distribution_function), which measures the system density as a function of distance from a particle.
The RDF is measured by calculating the average number of particles at a given distance from each particle and then averaging that measure over all particles.
In a perfect crystal, the RDF should be just a set of [delta functions](https://en.wikipedia.org/wiki/Delta_function) since all particles are located at precise fixed distances from one another.
In systems of particles like the ones we are simulating, we instead expect to see smoother sets of peaks.

Below, we calculate the RDF by binning all particles using `freud` (see [the documentation](https://freud.readthedocs.io/en/stable/density.html#freud.density.RDF) for more information).

In [ ]:
@interact(frame=(1, num_frames))
def rdf_demo(frame=num_frames):
    with gsd.hoomd.open(JOB.fn('trajectory.gsd')) as traj:
        frame = traj[frame]
            
        box = freud.box.Box.from_box(frame.configuration.box[:2].tolist())
        rdf = freud.density.RDF(np.sqrt(box.Lx**2 + box.Ly**2)/5, box.Lx/1000)
        rdf.compute(box, frame.particles.position)

    fig, ax = plt.subplots(dpi=DPI)
    ax.set_xlabel(r'r [$\sigma$]')
    ax.set_ylabel(r'$g(r)$ [-]')
    ax.plot(rdf.R, rdf.RDF)

Both fluids and solids will show a few strong peaks as there are short range correlations. Where they differ:
* **Fluid:** g(r) will tend toward 1.0 at moderate and large values of r.
* **Solid:** g(r) will show oscillations out to large values of r. Solids take some time to develop, so early frames may show fluid-like behavior while later frames develop longer range correlations.

However, as it is a function of distance alone, the RDF is clearly a limited way of characterizing our system, which we know should exhibit some ordering that depends on the angle between particles.

### Potential of Mean Force and Torque (PMFT)

To get a more informative picture, we can look at the potential of mean force and torque (PMFT).
The PMFT is a generalization of the classical [potential of mean force](https://en.wikipedia.org/wiki/Potential_of_mean_force) (PMF), which measures the average potential energy surface about each particle in the system as a function of distance.
The PMF can be calculated directly as $w(r) = -\beta \log(g(r))$ (where $g(r)$ is the radial distribution function).

The PMFT takes this one step further, looking at the potential energy surface as a function of both distance and angle.
As a result, the PMFT can capture the differences in a potential energy surface induced by, for instance, the shape of a particle.
In practice, the PMFT is calculated by [binning space and counting particles](https://freud.readthedocs.io/en/stable/pmft.html#freud.pmft.PMFTXY2D).
We plot the results below.

In [ ]:
@interact(frame=(1, num_frames))
def pmft_demo(frame=num_frames):
    with gsd.hoomd.open(JOB.fn('trajectory.gsd')) as traj:
        frame = traj[frame]
            
        box = freud.box.Box.from_box(frame.configuration.box[:2].tolist())
        pmft = freud.pmft.PMFTXY2D(4, 4, 300, 300)
        pmft.compute(box, frame.particles.position, quat2ang(frame.particles.orientation))

    fig, ax = plt.subplots(1, 1, figsize=(10, 8), dpi=DPI)
    draw_pmft(fig, ax, pmft, JOB.sp.n)

Looking at early frames of our simulation, we see that the PMFT looks almost circularly uniform.
In fact, in this case the PMFT shows the same information as the RDF.
There is a low energy ring immediately surrounding the central particle, and past that we see a mostly uniform potential energy surface at all distances.
The PMFT doesn't tell us much about the fluid-solid transition, but does help explain how the effective interactions between polygons changes.

* **Low vertex polygons**: In later frames, however, we start to see more features.
In particular, the PMFT shows smaller energy wells near the edges of the polygon, indicating polygons like to sit edge to edge. 
* **High vertex polygons**: In later frames, you should still see circular rings. These polygons are close enough to circles that they don't feel there edges strongly.

Note: The system sizes in this tutorial are very small and the PMFTs are only obtained from a single frame. For much cleaner PMFT data, one should use larger system sizes and average over many frames. See the paper [Shape and Symmetry Determine Two-Dimensional Melting Transitions of Hard Regular Polygons](http://dx.doi.org/10.1103/PhysRevX.7.021001) for well-sampled PMFTs of polygons.

This is additional information not present in the orientationally averaged RDF.

## Identifying the Fluid-Solid Transition

Finally, we return to our original problem: finding the pressure at which the system undergoes a phase transition.
We plot the average value of the hexatic order parameter (in the final $N$ frames of our simulation) as a function of pressure, and we look for a pressure at which we see a sudden jump in the order parameter.

We will store the identified transition pressures in a dict as part of the *project document*.
The *project document*, like the *job document* is a key-value storage, but central to the project.
It is ideal for data that does not directly belong to a specific job.

In [ ]:
betaP_transitions = project.doc.setdefault('betaP_transition', dict())

### Averaged Hexatic Order Parameter

Next, we define a function that calculates the average hexatic order parameter for a set of jobs.

In [ ]:
from scipy.stats import sem

def compute_average_hexatic_order_parameter(jobs, num_frames):
    hops = []
    for job in jobs:
        if job.isfile('trajectory.gsd'):
            with gsd.hoomd.open(job.fn('trajectory.gsd')) as traj:
                for frame in traj[-num_frames:]:
                    box = freud.box.Box.from_box(frame.configuration.box[:2].tolist())
                    hop = freud.order.HexOrderParameter(rmax=1.2, k=6)
                    try:
                        hop.compute(box, frame.particles.position)
                        hops.append(np.absolute(np.mean(hop.psi)))
                    except RuntimeError as error:
                        print("{}: {}".format(job, error), file=sys.stderr)
        else:
            print("Missing data:", job, file=sys.stderr)
    return np.array(hops) if hops else np.nan

Then we implement an **interactive** plot that allows us to 

 1. display the average hexatic order parameter at the simulated pressures,
 2. initialize new jobs at selected pressures, and
 3. save the selected pressure as the transition pressure.
 

 * To initialize new jobs, move the slider to the desired pressure and click the `[Init new jobs]` button.
 * To store a transition pressure, move the slider to the desired pressure and click on the `[Store transition pressure]` button.

**The following cell contains a lot of code that is concerned with the logic of setting up the interactive plot.
It is not crucial to follow all steps below!**

In [ ]:
from matplotlib import pyplot as plt
from ipywidgets import widgets, interactive
from IPython.display import display, clear_output, Markdown
from init import init_jobs
from multiprocessing import Process

# We set up one widget that allows us to select the number of vertices (n)
# and one widget that controls the pressure slider.
n_widget = widgets.Dropdown(options=list(project.detect_schema()['n'][int]))
betaPs = project.detect_schema()['betaP'][float]
betaP_widget = widgets.FloatSlider(value=min(betaPs), min=min(betaPs), max=max(betaPs), step=0.1)
replica_widget = widgets.IntSlider(min=2, max=9, value=3)


# Since we might have simulated different ranges of pressure for each number
# of vertices, we update the pressure slider dynamically.
def set_betaP_range(arg, num_frames=5):
    betaPs = {job.sp.betaP for job in project.find_jobs({'n': arg['new']})}
    betaP_widget.min, betaP_widget.max = min(betaPs), max(betaPs)
    betaP_widget.value = min(betaPs)
n_widget.observe(set_betaP_range, 'value')


def plot_hexatic_order_parameter_vs_pressure(n=0, betaP_select=None, replicas=3, num_frames=5):
    """Plot the hexatic order parameter versus the pressure.
    
    This is an interactive plot that allows users to initialize new simulation jobs
    for the selected pressure or set the selected pressure as the transition pressure.
    """
    
    # Compute the average hexatic order parameter (hop) as a function of the pressure.
    # This function utilizes job selection with find_jobs() and a grouping operation
    # groupby() in combination with a dict-comprehension.
    hops = {betaP: compute_average_hexatic_order_parameter(group, num_frames)
            for betaP, group in project.find_jobs(dict(n=n)).groupby('betaP')}
    
    # Plot the order parameter versus pressure.
    fig, ax = plt.subplots(dpi=DPI)
    ax.set_title('$n={}$'.format(n))
    
    means = [np.mean(x) for x in hops.values()]
    stderr = [sem(x) for x in hops.values()]
    ax.errorbar(hops.keys(), means, stderr, marker='o', linestyle='None')
    ax.set_xlabel(r'Pressure $\beta P$')
    ax.set_ylabel('Average hexatic order parameter')
    
    # Identified any missing values and display a blue dotted line where no data is available.
    x_nans = [x for x in hops if np.isnan(hops[x]).any()]
    for x in x_nans:
        ax.axvline(x=x, c='#1f77b4', ls='--', alpha=1.0)
    if x_nans:
        display(Markdown("WARNING: Some data is missing. Make sure to execute all jobs for "
                "example by executing `./project.py run --parallel` on the command line or "
                "the cell below."))
        

    # Get the transition pressure for the given n.
    # Note: Keys in JSON are *always* strings, even if they represent numbers.
    betaP_stored = betaP_transitions.get(str(n))

    if betaP_stored is None:
        # No transition pressure stored yet.
        ax.text(0.5, 0.5, 'No transition pressure stored yet.', transform=ax.transAxes, horizontalalignment='center')
    else:
        # Plot the (stored) transition pressure as a red solid line.
        ax.axvline(x=betaP_stored, c='red')
        ax.text(betaP_stored - 0.2, 0.2, 'stored', rotation=90, transform=ax.get_xaxis_transform())    
    
    if betaP_stored is None or betaP_stored != betaP_select:
        # Plot the selected pressure as a dashed red line.
        ax.axvline(x=betaP_select, color='red', ls='-.', alpha=0.5)
        ax.text(betaP_select - 0.2, 0.2, 'selected', rotation=90, transform=ax.get_xaxis_transform())
    
    plt.show()

    
# Convert the plotting function into an interactive function.
f = interactive(plot_hexatic_order_parameter_vs_pressure,
                n=n_widget, betaP_select=betaP_widget, replicas=replica_widget, num_frames=(1, 10))


# Setup the buttons for the interface below
def save_transition_pressure(button):
    "Store the selected pressure in the project document."
    n, betaP = n_widget.value, betaP_widget.value
    project.doc.setdefault('betaP_transition', dict())[str(n)] = betaP
    display("Transition pressure for n={} set to {}.".format(n, betaP))
    clear_output(wait=True)
    f.update()
button_save = widgets.Button(description='Store transition pressure')
button_save.on_click(save_transition_pressure)


def init_new_jobs(button):
    "Initialize new jobs for the selected pressure."
    init_jobs(project, n=n_widget.value, betaP=betaP_widget.value, num_replicas=replica_widget.value)
    display("Initialized new jobs for pressure {}.".format(betaP_widget.value))
    clear_output(wait=True)
    f.update()
button_init = widgets.Button(description='Init new job(s)')
button_init.on_click(init_new_jobs)

# Display the interactive plot and the three buttons directly below it.
display(widgets.HBox((button_init, button_save)), f)

If you execute the below cell initially, you'll see that for all the jobs in the workspace we have already run the simulation and computed $\psi$.
Once you have added a few new jobs, you can run the below cell again to see that the new jobs are not yet complete, as is evidenced by the "operation" column indicating that there is something to run and the "labels" column not yet indicating that $\psi$ has been computed.

In [ ]:
project.print_status(detailed=True, parameters=('n', 'betaP'))

To run jobs, you can simply execute the subsequent cell:

In [ ]:
project.run(np=2, progress=True)

## The Phase Diagram

After we have identified  transition pressures for the different shapes, we can plot them all together in one diagram.

You might want to initialize more simulations with the `./init.py` script for other vertices (*e.g.* `./init.py 6`).

In [ ]:
from IPython.display import HTML

ns = list(sorted(project.doc.betaP_transition))
betaPs = [betaP_transitions[n] for n in ns]

if len(ns):
    fig, ax = plt.subplots(dpi=DPI)
    ax.plot(ns, betaPs, 'o-')
    ax.set_xlabel('# Vertices ($n$)')
    ax.set_ylabel(r"Transition Pressure ($\beta P$)")
    ax.set_title("Transition pressure diagram")
    plt.show()
else:
    display(HTML('<font color="red">No transition pressures stored yet.</font>'))